# Darcy equation

In this tutorial we present how to solve a Darcy equation with [PyGeoN](https://github.com/compgeo-mox/pygeon) in themoving domain case (the upper boundary will move).  The unkwons are the velocity $u$, the elevation head $h$ and the height of the upper boundary $\eta$.

Let $\Omega=(0,1)\times(0,\eta)$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$K$ the matrix permeability, we want to solve the following problem: find $(\bm{u}, h)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
K^{-1} {\bm{u}} + \nabla h = {0}\\
S_s \frac{\partial{h}}{\partial t} + \nabla \cdot {u} = f
\end{array}
&\text{in } \Omega
\end{array}
\right.
$$

In order to solve the problem, we will perfom a change of coordinates to a reference domain $\hat{\Omega}=(0,1)^2$ through the (linear) trasnformation $R : \Omega \rightarrow \hat{\Omega}$ (and its inverse function $D : \hat{\Omega} \rightarrow \Omega$).
Recall that $\hat{\nabla}R=(\nabla D)^{-1}$.

Let $\hat{h}$ and $\hat{\bm{u}}$ be $h$ and $\bm{u}$ respectevely in the reference domain and let $\hat{K}$ be the transformed permeability matrix, defined as $\hat{K}=det(\hat{\nabla}D) (\hat{\nabla} D)^{-1} K (\hat{\nabla} D)^{-T}$.

The equation describing the motion of $\partial_{top}\Omega$ is:
$$

\phi \frac{\partial \eta}{\partial t} = \hat{u_3}

$$

The transformed equations in $\hat{\Omega}$ is:
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\hat{K}(\hat{h})^{-1} {\hat{u}} + \hat{\nabla} \hat{h} = {0}\\
\hat{S}_s \frac{\partial{\hat{h}}}{\partial t} + \hat{\nabla} \cdot {\hat{\bm{u}}} = f
\end{array}
&\text{in } \hat{\Omega}
\end{array}
\right.
$$
with boundary conditions:
$$ \hat{h} = \eta \text{ on } \Gamma \qquad \hat{h} = \ell \text{ on } \Gamma_D \qquad \hat{\bm{\nu}} \cdot \hat{\bm{u}} = 0 \text{ on } \Gamma_N$$

The weak formulation will be:
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\int_{\Omega}\hat{K}(\hat{h})^{-1} {\bm{\hat{u}}} \cdot \bm{v} \, d\Omega + \int_{\Omega} \hat{\bm{\nabla}} \hat{h} \cdot \bm{v} \, d\Omega = {0}\\
\int_{\Omega} \hat{S}_s \frac{\partial{\hat{h}}}{\partial t} v \, d\Omega - \int_{\Omega} \bm{\hat{u}} \cdot \bm{\nabla} v \, d\Omega = \int_{\Omega} fv \, d\Omega - \int_{\partial\Omega} v \bm{u} \cdot \bm{\nu} d\sigma\\
\int_{\Gamma} \phi \frac{\partial \eta}{\partial t} v \, d\sigma = \int_{\Gamma} \hat{u_3} v \, d\sigma
\end{array}
\end{array}
\right.
$$

In order to reduce the number of required equations, we can observe that $\eta = \hat{h} |_{\Gamma}$. 
Thus, we can reduce the problem to solve:

$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\int_{\Omega}\hat{K}(\hat{h})^{-1} {\bm{\hat{u}}} \cdot \bm{v} \, d\Omega + \int_{\Omega} \hat{\bm{\nabla}} \hat{h} \cdot \bm{v} \, d\Omega = {0}\\
\int_{\Omega} \hat{S}_s \frac{\partial{\hat{h}}}{\partial t} v \, d\Omega - \int_{\Omega} \bm{\hat{u}} \cdot \bm{\nabla} v \, d\Omega = \int_{\Omega} fv \, d\Omega - \int_{\Gamma} \phi \frac{\partial\hat{h}}{\partial t} v d\sigma\\
\hat{h}=\ell \quad \text{on } \Gamma_D
\end{array}
\end{array}
\right.
$$

In [1]:
import numpy as np
import scipy.sparse as sps
import os 
import shutil

import porepy as pp
import pygeon as pg

In [2]:
output_directory = 'output'

We create now the grid, since we will use a Raviart-Thomas approximation for ${q}$ we are restricted to simplices. In this example we consider a 2-dimensional structured grid, but the presented code will work also in 1d and 3d. PyGeoN works with mixed-dimensional grids, so we need to convert the grid.

In [3]:
N = 5
sd = pp.StructuredTriangleGrid([N] * 2, [1] * 2)
# convert the grid into a mixed-dimensional grid
mdg = pp.meshing.subdomains_to_mdg([sd])

With the following code we set the data, in particular the permeability tensor and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [4]:
key = "flow"
bc_val = []
bc_ess = []

velocity_discretization_field = pg.RT0(key)
pressure_discretization_field = pg.PwConstants(key)

for sd, data in mdg.subdomains(return_data=True):
    # permeability tensor
    perm = pp.SecondOrderTensor(np.ones(sd.num_cells))
    parameters = {
        "second_order_tensor": perm,
    }
    pp.initialize_data(sd, data, key, parameters)
    
    # with the following steps we identify the portions of the boundary
    # to impose the boundary conditions
    left_right = np.logical_or(sd.face_centers[0, :] == 0,  sd.face_centers[0, :] == 1)

    bottom = sd.face_centers[1, :] == 0
    top    = sd.face_centers[1, :] == 1

    ess_p_dofs = np.zeros(pressure_discretization_field.ndof(sd), dtype=bool)

    def h_bc(x): return 1

    bc_val.append(-velocity_discretization_field.assemble_nat_bc(sd, h_bc, bottom))
    bc_ess.append(np.hstack((left_right, ess_p_dofs)))

Once the data are assigned to the mixed-dimensional grid, we construct the matrices. In particular, the linear system associated with the equation is given as
$$
\left(
\begin{array}{cc} 
M & B^\top\\
B & 0
\end{array}
\right)
\left(
\begin{array}{c} 
q\\ 
p
\end{array}
\right)
=\left(
\begin{array}{c} 
p_{\partial}\\ 
0
\end{array}
\right)
$$<br>
where $p_{\partial}$ is the vector associated to the pressure boundary contions. To construct the saddle-point problem, we rely on the `scipy.sparse` function `bmat`. Once the matrix is created, we also construct the right-hand side containing the boundary conditions.

In [5]:
# construct the local matrices
mass = pg.face_mass(mdg)
div = pg.cell_mass(mdg, pressure_discretization_field) * pg.div(mdg)

# assemble the saddle point problem
spp = sps.bmat([[mass, -div.T], 
                [ div,  None]], format="csc")

# get the degrees of freedom for each variable
dof_p, dof_q = div.shape

# assemble the right-hand side
rhs = np.zeros(dof_p + dof_q)
rhs[:dof_q] += np.hstack(bc_val)

In [9]:
dof_p, dof_q

(50, 85)

We need to solve the linear system, PyGeoN provides a framework for that. The actual imposition of essential boundary conditions (flux boundary conditions) might change the symmetry of the global system, the class `pg.LinearSystem` preserves this structure by internally eliminating these degrees of freedom. Once the problem is solved, we extract the two solutions $q$ and $p$.

In [6]:
# solve the problem
ls = pg.LinearSystem(spp, rhs)
ls.flag_ess_bc(np.hstack(bc_ess), np.zeros(dof_q + dof_p))
x = ls.solve()

# extract the variables
q = x[:dof_q]
p = x[-dof_p:]

Since the computed $q$ is one value per facet of the grid, for visualization purposes we project the flux in each cell center as vector. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [7]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [8]:
# post process variables
proj_q = velocity_discretization_field.eval_at_cell_centers(sd)
cell_q = (proj_q * q).reshape((3, -1), order="F")
cell_p = pressure_discretization_field.eval_at_cell_centers(sd) * p

for _, data in mdg.subdomains(return_data=True):
    data[pp.STATE] = {"cell_q": cell_q, "cell_p": cell_p}

save = pp.Exporter(mdg, "sol", output_directory)
save.write_vtu(["cell_q", "cell_p"])

NameError: name 'RT0' is not defined